In [11]:
import pandas as pd
import math

In [30]:
df = pd.read_hdf('/home/gublu/Desktop/THINKSTATS/Competition/hdf/maya.h5', 'data')
#tr = df = pd.read_hdf('/home/gublu/Desktop/THINKSTATS/Competition/hdf/application_train.h5', 'data')

In [31]:
# class
class WOEScores:

  def __init__(self, df):
    self.df = df
    self.ids = self.df['SK_ID_CURR']
    self.df = self.df.drop('SK_ID_CURR', 1)
    self.woe_df = pd.DataFrame()
    self.bin_the_df()
    self.woe_encoded_df()

  def append_woe_scores(self):
    self.split_cols_tablewise_and_calculate_int_score()
    return (self.df, self.woe_df)


  def split_cols_tablewise_and_calculate_int_score(self):
    b_cols = [col for col in list(self.woe_df) if col.startswith('B_')]
    cc_cols = [col for col in list(self.woe_df) if col.startswith('CC_')]
    ip_cols = [col for col in list(self.woe_df) if col.startswith('IP_')]
    pa_cols = [col for col in list(self.woe_df) if col.startswith('PA_')]
    pos_cols = [col for col in list(self.woe_df) if col.startswith('POS_')]
    all_but_tr_te_cols = b_cols + cc_cols + ip_cols + pa_cols + pos_cols
    tr_te_cols = list(set(list(self.woe_df)) - set(all_but_tr_te_cols))
    self.df['WOE_SCORE_B']  = self.woe_df[b_cols].sum(axis=1)
    self.df['WOE_SCORE_CC']  = self.woe_df[cc_cols].sum(axis=1)
    self.df['WOE_SCORE_IP']  = self.woe_df[ip_cols].sum(axis=1)
    self.df['WOE_SCORE_PA']  = self.woe_df[pa_cols].sum(axis=1)
    self.df['WOE_SCORE_POS']  = self.woe_df[pos_cols].sum(axis=1)
    self.df['WOE_SCORE_ALL_BUT_TR_TE']  = self.woe_df[all_but_tr_te_cols].sum(axis=1)
    self.df['WOE_SCORE_TR_TE']  = self.woe_df[tr_te_cols].sum(axis=1)


  def bin_the_df(self):
    categorical_columns = [col for col in list(self.df) if self.df[col].nunique()<=20]
    continuous_columns = [col for col in list(self.df) if self.df[col].nunique()>20]
    for col in continuous_columns:
      z = self.df[col].copy(deep=True)
      try:
        binned = pd.cut(z, 20)
      except ValueError as e:
        continue
      self.woe_df[col] = binned
    self.woe_df = pd.concat([self.woe_df, self.df[categorical_columns]], axis=1)
    self.woe_df['TARGET'] = df['TARGET']


  def woe_encoded_df(self):
    x_total_good = (self.woe_df['TARGET'] == 0).sum()
    x_total_bad = (self.woe_df['TARGET'] == 1).sum()
    for col in list(self.woe_df):
        if col == 'TARGET':
            continue
        bin_woes = []
        bin_labels = self.woe_df[col].unique()
        print(col)
        for bin_label in bin_labels:
            total = (self.woe_df[col] == bin_label).sum()
            good = ((self.woe_df[col] == bin_label) & (self.woe_df['TARGET'] == 0)).sum()
            good_percent = max(good / x_total_good, 1e-100) # to avoid 0 in woe calculation
            bad = ((self.woe_df[col] == bin_label) & (self.woe_df['TARGET'] == 1)).sum()
            bad_percent = max(bad /x_total_bad, 1e-100) # to avoid inf in woe calculation
            woe = round(math.log(good_percent/bad_percent), 5)
            bin_woes.append(woe)
        self.woe_df[col].replace(dict(zip(bin_labels, bin_woes)), inplace= True)


In [32]:
df, woe_df = WOEScores(df).append_woe_scores()

/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:133: RuntimeWarning: invalid value encountered in multiply
  y *= step
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:142: RuntimeWarning: invalid value encountered in add
  y += start
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:133: RuntimeWarning: invalid value encountered in multiply
  y *= step
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:133: RuntimeWarning: invalid value encountered in multiply
  y *= step
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:142: RuntimeWarning: invalid value encountered in add
  y += start
/home/gublu/Documents/Python/venv/lib/python3.6/site-packages/numpy/core/function_base.py:133: RuntimeWarning: invalid value encountered in multiply
  y *= step


AMT_CREDIT/AMT_ANNUITY
DAYS_BIRTH
EXT_SOURCE_3
EXT_SOURCE_2
EXT_SOURCE_1
AMT_CREDIT
AMT_ANNUITY
AMT_CREDIT/AMT_GOODS_PRICE
DAYS_ID_PUBLISH
DAYS_EMPLOYED/DAYS_BIRTH
OWN_CAR_AGE/DAYS_BIRTH
DAYS_REGISTRATION
AMT_ANNUITY/AMT_INCOME_TOTAL
REGION_POPULATION_RELATIVE
AMT_GOODS_PRICE
AMT_CREDIT/AMT_INCOME_TOTAL
TOTALAREA_MODE
AMT_INCOME_TOTAL/CNT_CHILDREN
AMT_INCOME_TOTAL/CNT_FAM_MEMBERS
AMT_INCOME_TOTAL
BASEMENTAREA_AVG
LIVINGAREA_MODE
YEARS_BEGINEXPLUATATION_AVG
LANDAREA_MODE
NONLIVINGAREA_AVG
INCOME/CNT_FAM_MEMBERS*AMT_CREDIT
NEW_SOURCES_PROD
DAYS_LAST_PHONE_CHANGE
OWN_CAR_AGE
AMT_INCOME_TOTAL/DAYS_BIRTH
AMT_CREDIT/DAYS_BIRTH
EXT_SCORES_STD
external_sources_weighted
EXT_SOURCES_min
EXT_SOURCES_max
EXT_SOURCES_sum
EXT_SOURCES_mean
EXT_SOURCES_nanmedian
PA_AMT_APPLICATION/AMT_CREDIT_MEAN
PA_AMT_APPLICATION/AMT_CREDIT_MAX
PA_CNT_PAYMENT_SUM
PA_CNT_PAYMENT_MEAN
PA_CNT_PAYMENT_VAR
PA_CNT_PAYMENT_MAX
PA_AMT_CREDIT/AMT_DOWN_PAYMENT_MEAN
PA_AMT_CREDIT/AMT_DOWN_PAYMENT_MAX
PA_AMT_APPLICATION_MAX
PA_

In [34]:
df.head()

,AMT_CREDIT/AMT_ANNUITY,DAYS_BIRTH,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,NEW_IS_DAYS_EMPLOYED_365243,AMT_CREDIT,AMT_ANNUITY,AMT_CREDIT/AMT_GOODS_PRICE,DAYS_ID_PUBLISH,...,int_score_x,int_score_y,all_woe_score,WOE_SCORE_B,WOE_SCORE_CC,WOE_SCORE_IP,WOE_SCORE_PA,WOE_SCORE_POS,WOE_SCORE_ALL_BUT_TR_TE,WOE_SCORE_TR_TE
0,16.453125,-9461,0.139404,0.262939,0.083008,0.0,406597.5,24700.5,1.158203,-2120,...,-1.212891,-1.21331,-8.86013,0.551451,0.032104,-0.81389,-1.72124,-0.65826,-2.609835,-14.989994
1,36.218750,-16765,NaN,0.622070,0.311279,0.0,1293502.5,35698.5,1.145508,-291,...,0.028763,0.02877,25.75990,224.921666,0.032104,-0.42909,6.23020,0.42458,231.179461,226.655046
2,20.000000,-19046,0.729492,0.556152,NaN,0.0,135000.0,6750.0,1.000000,-2531,...,-0.988770,-0.98895,2.17036,1.877936,0.032104,-0.94285,-0.28504,-0.56590,0.116251,4.850280
3,10.531250,-19005,NaN,0.650391,NaN,0.0,312682.5,29686.5,1.052734,-2437,...,0.048981,0.04898,-3.54679,-0.245094,-1.120879,-1.14481,-2.27563,-0.66931,-5.455723,4.740218
4,23.453125,-19932,NaN,0.322754,NaN,0.0,513000.0,21865.5,1.000000,-3458,...,0.554688,0.55464,2.52915,2.033646,0.032104,0.20549,0.16831,0.73225,3.171801,-2.503784
